<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 8 [Assessment]:** RAG Evaluation</font>

<br>

Welcome to the last notebook of the course! In the previous notebook, you integrated a vector store solution into a RAG pipeline! In this notebook, you will take that same pipeline and evaluate it using numerical RAG evaluation techniques incorporating LLM-as-a-Judge metrics!

<br>

### **Learning Objectives:**

- Learn how to integrate the techniques from prior notebooks to numerically approximate the goodness of your RAG pipeline.

- **Final Exercice**: ***By working through this notebook in the Course Environment,* you will be able to submit the coding component of the course!**

<br>

### **Questions To Think About:**

- As you go along, remember what our metrics actually represent. Should our pipeline pass these objectives? Is our judge LLM sufficient for evaluating the pipeline? Does a particular metric even matter for our use case?
- If we left the vectorstore-as-a-memory component in our chain, do you think it would still pass the evaluation? Additionally, is the evaluation useful for assessing vectorstore-as-a-memory performance? 

<br>

### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://www.nvidia.com/en-sg/training/instructor-led-workshops/building-rag-agents-with-llms/). If sharing this material, please give credit and link back to the original course.

<br>

### **Environment Setup:**

In [2]:
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

NVIDIAEmbeddings.get_available_models(base_url="http://llm_client:9000/v1")
embedder = NVIDIAEmbeddings(
    model="nvidia/embed-qa-4", truncate="END",
    base_url="http://llm_client:9000/v1"
)

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    base_url="http://llm_client:9000/v1"
)

----

<br>

## **Part 1:** Pre-Release Evaluation

In our previous notebook, we successfully combined several concepts to create a document chatbot with the aim of responsive and informative interactions. However, the diversity of user interactions necessitates comprehensive testing to truly understand the chatbot's performance. Thorough testing in varied scenarios is crucial to ensure that the system is not only robust and versatile but also aligns with user and provider expectations.

After defining your chatbot's roles and implementing the necessary features, evaluating it becomes a multi-stage process:

- **Typical Use Inspection:** Start by testing scenarios most relevant to your use case. See if your chatbot can reliably navigate discussions with limited human intervention.

    - Additionally, identify limitations or compartments that should be redirected to a human for inspection/supervision (i.e., human swap-in to confirm transactions or perform sensitive navigation) and implement those options.

- **Edge Case Inspection:** Explore the boundaries of typical use, identifying how the chatbot handles less common but plausible scenarios.

    - Before any public release, assess critical boundary conditions that could pose liability risks, such as the potential generation of inappropriate content.

    - Implement well-tested guardrails on all outputs (and possibly inputs) to limit undesired interactions and redirect users into predictable conversation flows.

- **Progressive Rollout:** Rolling out your model to a limited audience (first internal, then [A/B](https://en.wikipedia.org/wiki/A/B_testing)) and implement analytics features like usage analytics dashboards and feedback avenues (flag/like/dislike/etc).

Of these three steps, the first two can be done by a small team or an individual and should be iterated on as part of the development process. Unfortunately, this needs to be done frequently and can be prone to human error. **Luckily for us, LLMs can be used to help out with LLM-as-a-Judge formulations!**

*(Yeah, this probably isn't surprising by now. LLMs being strong is why this course is here...).*

----

<br>

## **Part 2:** LLM-as-a-Judge Formulation

In the realm of conversational AI, using LLMs as evaluators or 'judges' has emerged as a useful approach for configurable automatic testing of natural language task performance:

- An LLM can simulate a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on the synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**There are several popular frameworks for off-the-shelf judge formulations including:**
- [**RAGAs (short for RAG Assessment)**](https://docs.ragas.io/en/stable/), which offers a suite of great starting points for your own evaluation efforts.
- [**LangChain Evaluators**](https://python.langchain.com/docs/guides/evaluation/), which are similar first-party options with many implicitly-constructible agents.

Instead of using the chains as-is, we will instead expand on the ideas and evaluate our system with a more custom solution.

----

<br>

## **Part 3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/docs/guides/evaluation/examples/comparisons). 

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***





**This will be the system used for the final evaluation!** To see how this system is integrated into the autograder, please check out the implementation in [`frontend/server_app.py`](frontend/server_app.py).

<br>

### **Task 1:** Pull In Your Document Retrieval Index

For this exercise, you will pull in the `docstore_index` file you created as part of your earlier notebook. The following cell should be able to load in the store as-is.

In [4]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


Constructed aggregate docstore with 238 chunks

Sample Chunk:

Paper: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

Summary: Large pre-trained language models have been shown to store factual knowledge
in their parameters, and achieve state-of-the-art results when fine-tuned on
downstream NLP tasks. However, their ability to access and precisely manipulate
knowledge is still limited, and hence on knowledge-intensive tasks, their
performance lags behind task-specific architectures. Additionally, providing
provenance for their decisions and updating their world knowledge remain open
research problems. Pre-trained models with a differentiable access mechanism to
explicit non-parametric memory can overcome this issue, but have so far been
only investigated for extractive downstream tasks. We explore a general-purpose
fine-tuning recipe for retrieval-augmented generation (RAG) -- models which
combine pre-trained parametric and non-parametric memory for language
generation. We introduce RAG models where the parametric memory is a
pre

<br>

### **Task 2: [Exercise]** Pull In Your RAG Chain

Now that we have our index, we can recreate the RAG agent from the previous notebook! 

**Key Modifications:**
- To keep things simple, feel free to disregard the vectorstore-as-a-memory component. Incorporating it will require some more overhead and will make the exercise a bit more complicated.

In [41]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder
from langserve import add_routes
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from fastapi import FastAPI
from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models(base_url="http://llm_client:9000/v1")
embedder = NVIDIAEmbeddings(
    model="nvidia/embed-qa-4", truncate="END",
    base_url="http://llm_client:9000/v1"
)

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    base_url="http://llm_client:9000/v1"
)
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary

## Chain 1 Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str  ## TODO
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller
generator_chain = RunnableAssign({"output" : chat_prompt | llm })  ## TODO
generator_chain = {'output' : generator_chain} | RunnableLambda(output_puller)  ## GIVEN

## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    rag_chain,
    path="/generator",
)

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

{'input': 'Tell me something interesting!', 'context': '[Quote from Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena] .\\nThis paper makes two contributions: (1) a systematic study of LLM-as-a-judge; and (2) human\\npreference datasets with high-quality questions and diverse user interactions from MT-bench and\\nChatbot Arena. In addition, we argue for the adoption of a hybrid evaluation framework for future\\nLLM benchmarks: by combining the existing capability-based benchmarks and the new preference-\\nbased benchmarks with LLM-as-a-judge, one can swiftly and automatically evaluate both the core\\ncapabilities and human alignment of models. We publicly release 80 MT-bench questions, 3K expert\\nvotes, and 30K conversations with human preferences for future study.\\nTable 1: Sample multi-turn questions in MT-bench.\\nCategory\\nSample Questions\\nWriting\\n1st Turn\\nCompose an engaging travel blog post about a recent trip to Hawaii, highlighting\\ncultural experiences and must-

<br>

### **Step 3:** Generating Synthetic Question-Answer Pairs

In this section, we can implement the first few part of our evaluation routine:

- **Sample the RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [42]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_template('INSTRUCTION:\n\n{system}\n\nINPUT:\n\n{input}')

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How does Mistral 7B improve performance and efficiency compared to other language models, such as Llama 2
13B and Llama 1 34B? Additionally, how can large language model (LLM) based chat assistants be effectively 
evaluated using strong LLMs as judges?

Answer: Mistral 7B outperforms Llama 2 13B across all evaluated benchmarks and Llama 1 34B in reasoning, 
mathematics, and code generation. It leverages grouped-query attention (GQA) for faster inference and sliding 
window attention (SWA) to handle sequences of arbitrary length with reduced inference cost. The model's 
architecture includes a transformer with the following parameters: dim (4096), n_layers (32), head_dim (128), 
hidden_dim (14336), n_heads (32), n_kv_heads (8), window_size (4096), context_len (8192), and vocab_size (32000).

QA Pair 2

Question: How does the proposed Transformer model in "Attention Is All You Need" differ from the existing sequence 
transduction models, and what improvements does it bring to machine translation tasks? How well does it generalize 
to other tasks like English constituency parsing?

Answer: The proposed Transformer model is a new simple network architecture based solely on attention mechanisms, 
dispensing with recurrence and convolutions entirely. Unlike the dominant sequence transduction models which are 
based on complex recurrent or convolutional neural networks in an encoder-decoder configuration, the Transformer 
model connects the encoder and decoder through an attention mechanism. Experiments on two machine translation 
tasks, WMT 2014 English-to-German and English-to-French translation tasks, show that the Transformer models are 
superior in quality while being more parallelizable and requiring significantly less time to train. The Transformer
model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best 
results by over 2 BLEU, and establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 
3.5 days on eight GPUs for the English-to-French translation task. The Transformer generalizes well to other tasks 
such as English constituency parsing with large and limited training data.

QA Pair 3

Question: What are the inherent limitations of current Large Language Models (LMs) and how does the proposed 
neuro-symbolic architecture, the Modular Reasoning, Knowledge and Language (MRKL) system, aim to address these 
limitations? How does this system compare in performance to other sequence transduction models like the Transformer
model?

Answer: The inherent limitations of current Large Language Models (LMs) are that despite their versatility and 
impressive capabilities, their output is often wrong or ridiculously incorrect. This is exemplified by the sample 
output of GPT-3 on simple queries. The proposed neuro-symbolic architecture, the Modular Reasoning, Knowledge and 
Language (MRKL) system, aims to address these limitations by adopting a systems approach that involves knowledge 
and reasoning in addition to linguistic processing. The MRKL system is a flexible architecture with multiple neural
models, complemented by discrete knowledge and reasoning modules. In terms of performance, the Transformer model, a
sequence transduction model based solely on attention mechanisms, has been shown to be superior in quality while 
being more parallelizable and requiring significantly less time to train. However, the MRKL system's performance is
not compared to the Transformer model in the provided document.

<br>

### **Step 4:** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [48]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    rag_answer_tokens = list(rag_chain.stream(q))
    rag_answer = "".join(token['output'] for token in rag_answer_tokens if 'output' in token)
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

QA Pair 1
Question: How does Mistral 7B improve performance and efficiency compared to other language models, such as Llama 2
13B and Llama 1 34B? Additionally, how can large language model (LLM) based chat assistants be effectively 
evaluated using strong LLMs as judges?

RAG Answer: Based on the provided document, Mistral 7B improves performance and efficiency compared to other 
language models such as Llama 2 13B and Llama 1 34B in several ways.

Firstly, Mistral 7B is a language model with 7 billion parameters that is engineered for superior performance and 
efficiency. It outperforms the best open 13B model (Llama 2) across all evaluated benchmarks and the best released 
34B model (Llama 1) in reasoning, mathematics, and code generation (Source: Mistral 7B).

Secondly, Mistral 7B leverages grouped-query attention (GQA) for faster inference, and sliding window attention 
(SWA) to effectively handle sequences of arbitrary length with a reduced inference cost. This makes it more 
efficient than other models (Source: Mistral 7B).

As for evaluating large language model (LLM) based chat assistants, it can be challenging due to their broad 
capabilities and the inadequacy of existing benchmarks in measuring human preferences. However, the usage of strong
LLMs as judges to evaluate these models on more open-ended questions has been explored. This method, known as 
LLM-as-a-judge, examines the usage and limitations of this approach, including position, verbosity, and 
self-enhancement biases, as well as limited reasoning ability. Solutions to mitigate some of these issues have also
been proposed (Source: Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena).

QA Pair 2
Question: How does the proposed Transformer model in "Attention Is All You Need" differ from the existing sequence 
transduction models, and what improvements does it bring to machine translation tasks? How well does it generalize 
to other tasks like English constituency parsing?

RAG Answer: The Transformer model proposed in "Attention Is All You Need" differs from existing sequence 
transduction models because it is the first transduction model that relies entirely on self-attention to compute 
representations of its input and output, without using sequence-aligned RNNs (Recurrent Neural Networks) or 
convolution. Most competitive neural sequence transduction models have an encoder-decoder structure, where the 
encoder maps an input sequence of symbol representations to a sequence of continuous representations, and then the 
decoder generates an output sequence of symbols one element at a time.

The Transformer model replaces the recurrent layers most commonly used in encoder-decoder architectures with 
multi-headed self-attention. This architecture allows the model to be trained significantly faster than 
architectures based on recurrent or convolutional layers. The model has been applied to machine translation tasks 
and has achieved a new state of the art on both WMT 2014 English-to-German and WMT 2014 English-to-French 
translation tasks. In the former task, the best model outperforms even all previously reported ensembles.

The Transformer model also generalizes well to other tasks, such as English constituency parsing. The authors show 
that the model can be successfully applied to English constituency parsing both with large and limited training 
data. The model's performance on these tasks suggests that attention-based models have the potential to be applied 
to a wide range of tasks beyond machine translation.

Reference(s):
[Quote from Attention Is All You Need] . (2023). Attention Is All You Need. Retrieved from 
<https://arxiv.org/pdf/1706.03762.pdf>

Summary:
In "Attention Is All You Need," the authors propose the Transformer model, a new simple network architecture based 
solely on attention mechanisms, dispensing with recurrence and convolutions entirely. The model has been shown to 
be superior in quality while being more parallelizable and requiring significantly less time to train. It has 
achieved a new state of the art on machine translation tasks and has been successfully applied to English 
constituency parsing with both large and limited training data.

Source(s):
"Attention Is All You Need" (2023), A. Vaswani, N. Shazeer, N. Parmar, J. Uszkoreit, L. Jones, A. N. Gomez, L. 
Kaiser, and I. Polosukhin, <https://arxiv.org/pdf/1706.03762.pdf>

QA Pair 3
Question: What are the inherent limitations of current Large Language Models (LMs) and how does the proposed 
neuro-symbolic architecture, the Modular Reasoning, Knowledge and Language (MRKL) system, aim to address these 
limitations? How does this system compare in performance to other sequence transduction models like the Transformer
model?

RAG Answer: Great question! According to the paper "MRKL Systems: A modular, neuro-symbolic architecture that 
combines large language models, external knowledge sources and discrete reasoning" published on May 3, 2022, Large 
Language Models (LMs) are an essential backbone of any future AI system, but they have inherent limitations.

One limitation is that it is not practical to fine-tune and serve multiple large models. Furthermore, catastrophic 
forgetting makes it difficult to fine-tune a multi-task-trained LM on a new task that hadn't been covered in its 
training. This necessitates retraining on the entire task set, which is infeasible due to the cost of training such
models.

The proposed neuro-symbolic architecture, the Modular Reasoning, Knowledge and Language (MRKL) system, aims to 
address these limitations by adopting a systems approach. The MRKL system conceptualizes the challenge as one that 
involves knowledge and reasoning in addition to linguistic processing. It defines a flexible architecture with 
multiple neural models, complemented by discrete knowledge and reasoning modules. This approach enables the MRKL 
system to enjoy the benefits of self-supervised deep language models without suffering from the drawbacks mentioned
above.

Regarding the performance of the MRKL system compared to other sequence transduction models like the Transformer 
model, the paper does not provide a direct comparison. However, the MRKL system is designed to be more flexible and
modular than the Transformer architecture, which should enable it to handle a wider range of tasks and accommodate 
new tasks more easily. Additionally, the integration of external knowledge sources and discrete reasoning modules 
should enable the MRKL system to make more informed and logical decisions.

<br>

### **Step 5:** Implement A Human Preference Metric

In this section, we can implement the fourth part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [49]:
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION: 
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION: 
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How does Mistral 7B improve performance and efficiency compared to other language models, such 
as Llama 2 13B and Llama 1 34B? Additionally, how can large language model (LLM) based chat assistants be 
effectively evaluated using strong LLMs as judges?

Synth Answer: Answer: Mistral 7B outperforms Llama 2 13B across all evaluated benchmarks and Llama 1 34B in 
reasoning, mathematics, and code generation. It leverages grouped-query attention (GQA) for faster inference and 
sliding window attention (SWA) to handle sequences of arbitrary length with reduced inference cost. The model's 
architecture includes a transformer with the following parameters: dim (4096), n_layers (32), head_dim (128), 
hidden_dim (14336), n_heads (32), n_kv_heads (8), window_size (4096), context_len (8192), and vocab_size (32000).

RAG Answer: Based on the provided document, Mistral 7B improves performance and efficiency compared to other 
language models such as Llama 2 13B and Llama 1 34B in several ways.

Firstly, Mistral 7B is a language model with 7 billion parameters that is engineered for superior performance and 
efficiency. It outperforms the best open 13B model (Llama 2) across all evaluated benchmarks and the best released 
34B model (Llama 1) in reasoning, mathematics, and code generation (Source: Mistral 7B).

Secondly, Mistral 7B leverages grouped-query attention (GQA) for faster inference, and sliding window attention 
(SWA) to effectively handle sequences of arbitrary length with a reduced inference cost. This makes it more 
efficient than other models (Source: Mistral 7B).

As for evaluating large language model (LLM) based chat assistants, it can be challenging due to their broad 
capabilities and the inadequacy of existing benchmarks in measuring human preferences. However, the usage of strong
LLMs as judges to evaluate these models on more open-ended questions has been explored. This method, known as 
LLM-as-a-judge, examines the usage and limitations of this approach, including position, verbosity, and 
self-enhancement biases, as well as limited reasoning ability. Solutions to mitigate some of these issues have also
been proposed (Source: Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena).

Synth Evaluation: [2] The second answer is better than the first and does not introduce any inconsistencies.
Justification: The second answer not only includes the same information as the first answer regarding Mistral 7B's 
improvements in performance and efficiency compared to other language models, but it also provides additional 
context and explanation, such as the fact that Mistral 7B is a language model with 7 billion parameters and the 
reasoning behind using strong LLMs as judges for evaluating other large language models. The second answer also 
includes citations to its sources, providing additional credibility to the information presented.

Set 2

Question: Question: How does the proposed Transformer model in "Attention Is All You Need" differ from the existing
sequence transduction models, and what improvements does it bring to machine translation tasks? How well does it 
generalize to other tasks like English constituency parsing?

...

Synth Evaluation: [1] Justification: Both answers provide a thorough explanation of how the proposed Transformer 
model differs from existing sequence transduction models and highlight its improvements in machine translation 
tasks. However, Answer 1 is more precise in its details and provides specific BLEU scores achieved by the model in 
the WMT 2014 English-to-German and English-to-French translation tasks. Answer 2, while accurate, does not provide 
these specific details. Therefore, Answer 1 is more informative and specific, making it superior to Answer 2.

Set 3

Question: Question: What are the inherent limitations of current Large Language Models (LMs) and how does the 
proposed neuro-symbolic architecture, the Modular Reasoning, Knowledge and Language (MRKL) system, aim to address 
these limitations? How does this system compare in performance to other sequence transduction models like the 
Transformer model?

Synth Answer: Answer: The inherent limitations of current Large Language Models (LMs) are that despite their 
versatility and impressive capabilities, their output is often wrong or ridiculously incorrect. This is exemplified
by the sample output of GPT-3 on simple queries. The proposed neuro-symbolic architecture, the Modular Reasoning, 
Knowledge and Language (MRKL) system, aims to address these limitations by adopting a systems approach that 
involves knowledge and reasoning in addition to linguistic processing. The MRKL system is a flexible architecture 
with multiple neural models, complemented by discrete knowledge and reasoning modules. In terms of performance, the
Transformer model, a sequence transduction model based solely on attention mechanisms, has been shown to be 
superior in quality while being more parallelizable and requiring significantly less time to train. However, the 
MRKL system's performance is not compared to the Transformer model in the provided document.

RAG Answer: Great question! According to the paper "MRKL Systems: A modular, neuro-symbolic architecture that 
combines large language models, external knowledge sources and discrete reasoning" published on May 3, 2022, Large 
Language Models (LMs) are an essential backbone of any future AI system, but they have inherent limitations.

One limitation is that it is not practical to fine-tune and serve multiple large models. Furthermore, catastrophic 
forgetting makes it difficult to fine-tune a multi-task-trained LM on a new task that hadn't been covered in its 
training. This necessitates retraining on the entire task set, which is infeasible due to the cost of training such
models.

The proposed neuro-symbolic architecture, the Modular Reasoning, Knowledge and Language (MRKL) system, aims to 
address these limitations by adopting a systems approach. The MRKL system conceptualizes the challenge as one that 
involves knowledge and reasoning in addition to linguistic processing. It defines a flexible architecture with 
multiple neural models, complemented by discrete knowledge and reasoning modules. This approach enables the MRKL 
system to enjoy the benefits of self-supervised deep language models without suffering from the drawbacks mentioned
above.

Regarding the performance of the MRKL system compared to other sequence transduction models like the Transformer 
model, the paper does not provide a direct comparison. However, the MRKL system is designed to be more flexible and
modular than the Transformer architecture, which should enable it to handle a wider range of tasks and accommodate 
new tasks more easily. Additionally, the integration of external knowledge sources and discrete reasoning modules 
should enable the MRKL system to make more informed and logical decisions.

Synth Evaluation: [2] The second answer is better than the first and does not introduce any inconsistencies.
Justification: The second answer provides a more detailed and specific explanation of the inherent limitations of 
Large Language Models, citing the impracticality of fine-tuning and serving multiple large models and the challenge
of catastrophic forgetting. It also offers a more comprehensive description of how the proposed MRKL system aims to
address these limitations. While neither answer offers a direct comparison of the MRKL system's performance to the 
Transformer model, the second answer suggests that the MRKL system's flexibility and modularity may enable it to 
handle a wider range of tasks and make more informed decisions.

<br>

**Congratulations! We now have an LLM system that reasons about our pipeline and tries to evaluate it!** Now that we have some judge results, we can simply aggregate the results and see how often our formulation was according to an LLM:

In [50]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 0.6666666666666666


----

<br>

## **Part 4:** Advanced Formulations

The exercise above was meant to prepare you for the final assessment of the course and showcased a simple but effective evaluator chain. The objective and implementation details were provided for you, and the logic for using it probably makes sense now that you've seen it in action. 

With that being said, this metric was merely a product of us specifying:
- **What kind of behavior is important for our pipeline to have?**
- **What do we need to do in order to exhibit and evaluate this behavior?**

From these two questions, we could have come up with plenty of other evaluation metrics that could have assessed different attributes, incorporated different evaluator chain techniques, and even required different pipeline organization strategies. Though far from an exhaustive list, some common formulations you will likely come across may include:

- **Style Evaluation:** Some evaluation formulations can be as simple as "let me ask some questions and see if the output feels desirable." This might be used to see whether a chatbot "acts like it's supposed to" based on a description provided to a judge LLM. We're using quotations since this kind of assessment can reasonably be achieved with nothing but prompt engineering and a while loop.

- **Ground-Truth Evaluation:** In our chain, we used synthetic generation to create some random questions and answers using a sampling strategy, but in reality you may actually have some representative questions and answers that you need your chatbot to consistently get right! In this case, a modification of the exercise chain above should be implemented and closely monitored as you develop your pipelines.

- **Retrieval/Augmentation Evaluation:** This course made many assumptions about what kinds of preprocessing and prompting steps would be good for your pipelines, and much of this was determined by experimentation. Factors such as document preprocessing, chunking strategies, model selection, and prompt specification all played important roles, so creating metrics to validate these decisions may be of interest. This kind of metric might require your pipeline to output your context chunks or may even rely solely on embedding similarity comparisons, so keep this in mind when trying to implement a chain that works with multiple evaluation strategies. Consider the [**RagasEvaluatorChain**](https://docs.ragas.io/en/latest/howtos/integrations/langchain.html) abstraction as a decent starting point for making an custom generalizable evaluation routine. 

- **Trajectory Evaluation:** Using more advanced agent formulations, you can implement multiple-query strategies that assume the presence of conversational memory. With this, you can implement an evaluation agent which can:
    - Ask a series of questions in order to evaluate how well the agent is able to adapt and cater to the scenario. This kind of system generally considers a series of correspondence and aims to tease out and evaluate a "trajectory" of how the agent navigated the conversation. The [**LangChain Trajectory Evaluators documentation**](https://python.langchain.com/docs/guides/evaluation/trajectory/) is a good starting point.
    - Alternatively, you could also implement an evaluation agent that tries to achieve objectives by interacting with the chatbot. Such an agent can output whether they were able to navigate to their solution in a natural manner, and can even be used to generate a report about the percieved performance. The [**LangChain Agents documentation**](https://python.langchain.com/docs/modules/agents/concepts) is a good starting point!

<br>

At the end of the day, just make sure to use the tools you have at your disposal appropriately. By this point in the course, you should already be well-acquainted with the LLM core value propositions: **They're powerful, scalable, predictable, controllable, and orchestratable... but will act unpredictably when you just expect them to work by default.** Assess your needs, formulate and validate your pipelines, give enough information, and add as much control as you can to make your system work consistently, efficiently, and effectively.

----

<br>

## **Part 5: [Assessment]** Evaluating For Credit

Welcome to the last exercise of the course! Hopefully you've enjoyed the material and are ready to actually get credit for these notebooks! For this part:

- **Make sure you're in the course environment**
- **Make sure `docstore_index/` has been uploaded to the course environment...**
    - **...and contains [at least one Arxiv paper](https://arxiv.org/search/advanced) which has been updated recently.**
- **Make sure [`35_langserve.ipynb`](35_langserve.ipynb) is not occupying port 9012 with a running FastAPI service**

**Objective:** On launch, [**`frontend/frontend_block.py`**](frontend/frontend_block.py) had several lines of code which trigger the course pass condition. Once the service was deemed healthy, that sequence of code was replaced with `## secret` by another microservice. Your objective is to invoke that series of commands by using your pipeline to pass the **Evaluation** check! Recall [`35_langserve.ipynb`](35_langserve.ipynb) and use it as a starting example! As a recommendation, consider duplicating it so that you can keep the original as an authoritative reference. 

**Once Finished:** While your course environment is still open, please navigate back to your course environment launcher area and click the **"Assess Task"** button! After that, you're all done!

In [56]:
%%js
var url = 'http://'+window.location.host+'/8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

<IPython.core.display.Javascript object>

In [55]:
from langchain_core.prompts import ChatPromptTemplate
from langserve import RemoteRunnable
# Invoke generator route --> http://0.0.0.0:9012/generator
RemoteRunnable_generator = RemoteRunnable("http://0.0.0.0:9012/generator/")
RemoteRunnable_generator.invoke({"input": "Tell me something interesting"})

ConnectError: [Errno 111] Connection refused

----

<br>

## <font color="#76b900">**Congratulations On Completing The Course**</font>

Hopefully this course was not only exciting and challenging, but also adequately prepared you for work on the cutting edge of LLM and RAG system development! Going forward, you should have the skills necessary to tackle industry-level challenges and explore RAG deployment with open-source models and frameworks.

**Some NVIDIA-specific releases related to this that you may find interesting include:**
- [**NVIDIA NIMs**](https://www.nvidia.com/en-us/ai/), which offers microservice spinup routines that can be deployed on local compute.
- [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) is the current recommended framework for deploying GPU-accelerated LLM model engines in production settings.
- [**NVIDIA's Generative AI Examples Repo**](https://github.com/NVIDIA/GenerativeAIExamples), which includes the current canonical microservice example application and will be updated with new resources as new production workflows get released.
- [**The Knowledge-Based Chatbot Technical Brief**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief) which discusses additional publicly-accessible details on productionalizing RAG systems.

**Additionally, some key topics you may be interested in delving more into include:**
- [**LlamaIndex**](https://www.llamaindex.ai/), which has strong components that can augment and occasionally improve upon the LangChain RAG features.
- [**LangSmith**](https://docs.smith.langchain.com/), an upcoming agent productionalization service offered by LangChain.
- [**Gradio**](https://www.gradio.app/), though touched on in the course, has many more interface options which will be worth investigating. For inspiration, consider checking out [**HuggingFace Spaces**](https://huggingface.co/spaces) for examples.
- [**LangGraph**](https://python.langchain.com/docs/langgraph/) is a framework for graph-based LLM orchestration, and is a natural next step forward for those interested in [multi-agent workflows](https://blog.langchain.dev/langgraph-multi-agent-workflows/).
- [**DSPy**](https://github.com/stanfordnlp/dspy), a flow engineering framework that allows you to optimize LLM orchestration pipelines based on empirical performance results.

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>